In [1]:
import sys
project_root_path = '../..'
python_lib_path = project_root_path + '/python/lib'
sys.path.append(python_lib_path)
import commit_stats

In [2]:
import json
in_path = project_root_path + '/python/cfg/ds_pipeline.json'
config_map = json.loads(open(in_path, 'r').read())
config_map

cfg = config_map['language_analyzer'] # 99_problems

In [3]:

path = project_root_path + '/' + cfg['stats_dir']
all_commit_stats = commit_stats.load_commit_stats(cfg['stat_file_name'], path)
all_commit_stats[slice(0,2)]

[{'commit': '58451cf7106f02e70172ccce62592446877bfa9d',
  'Author': 'Jerome Kieffer <jerome.kieffer@terre-adelie.org>',
  'Date': '2022-06-02T15:05:54+00:00',
  'orig_timezone': 'UTC+02:00',
  'fileTypes': {'py': {'inserts': 2, 'deletes': 2, 'occurrences': 1}},
  'file_list': {'version.py\n': {'binary': 0, 'inserts': 2, 'deletes': 2}},
  'seen_in': ['silx-kit/fabio'],
  'author_hash': '652e1f8a490a871c08d8fcca60e9368e',
  'iso_datetime': '2022-06-02T15:05:54+00:00',
  'utc_datetime': '2022-06-02T15:05:54'},
 {'commit': 'b9834cbb1cc372623a26cb885b19ed3ad80a2228',
  'Author': 'Jerome Kieffer <jerome.kieffer@esrf.fr>',
  'Date': '2022-06-02T14:10:13+00:00',
  'orig_timezone': 'UTC+02:00',
  'fileTypes': {'py': {'inserts': 3, 'deletes': 3, 'occurrences': 2}},
  'file_list': {'setup.py\n': {'binary': 0, 'inserts': 2, 'deletes': 2},
   'version.py\n': {'binary': 0, 'inserts': 1, 'deletes': 1}},
  'seen_in': ['silx-kit/fabio'],
  'author_hash': '0aea9f0b6c4b48f4c17853b3fdc6b905',
  'iso_datet

In [4]:
ft_dicts = []
ft_dict_dict = {}
for stats in all_commit_stats:
    for file_type in stats['fileTypes']:
        commit_ft_dict = stats['fileTypes'][file_type]
        if file_type not in ft_dict_dict:
            ft_dict = {}
            ft_dicts.append(ft_dict)
            ft_dict_dict[file_type] = ft_dict
            ft_dict['type'] = file_type
            ft_dict['lines'] = 0
            ft_dict['files'] = 0
            ft_dict['commits'] = 0
            ft_dict['repos'] = {}
        ft_dict = ft_dict_dict[file_type]
        ft_dict['commits'] += 1
        ft_dict['lines'] += commit_ft_dict['inserts']
        ft_dict['files'] += commit_ft_dict['occurrences']
        #seen_in = commit_ft_dict['seen_in']
        #for project in seen_in:
        #    ft_dict['repos'][project] = 1


In [5]:
by_commit_count = sorted(ft_dicts, key=lambda d: d['commits'])
by_commit_count.reverse()
# if your odds of being in a commit are < 1/1000, you don't matter
main_type_stats = [stats for stats in by_commit_count if stats['commits'] > (len(all_commit_stats) / 1000)]
print(len(main_type_stats))
main_type_stats

62


[{'type': 'py',
  'lines': 132326032,
  'files': 4952578,
  'commits': 1949825,
  'repos': {}},
 {'type': 'rst',
  'lines': 13875165,
  'files': 644623,
  'commits': 381572,
  'repos': {}},
 {'type': 'noexttext',
  'lines': 17335969,
  'files': 416094,
  'commits': 218853,
  'repos': {}},
 {'type': 'js',
  'lines': 48497049,
  'files': 832320,
  'commits': 192608,
  'repos': {}},
 {'type': 'json',
  'lines': 72661386,
  'files': 1316392,
  'commits': 169400,
  'repos': {}},
 {'type': 'ts',
  'lines': 33151044,
  'files': 683450,
  'commits': 165842,
  'repos': {}},
 {'type': 'md',
  'lines': 8650540,
  'files': 357630,
  'commits': 149961,
  'repos': {}},
 {'type': 'c',
  'lines': 123019519,
  'files': 382766,
  'commits': 144556,
  'repos': {}},
 {'type': 'txt',
  'lines': 13724128,
  'files': 266142,
  'commits': 134302,
  'repos': {}},
 {'type': 'h',
  'lines': 25121694,
  'files': 451316,
  'commits': 133051,
  'repos': {}},
 {'type': 'yml',
  'lines': 1943171,
  'files': 140017,
 

In [ ]:
import pandas as pd

main_types = [stat['type'] for stat in main_type_stats]
main_types.index('json')
rows = []
for commit_stats in all_commit_stats:
    row = [0]*len(main_types)
    rows.append(row)
    for file_type in commit_stats['fileTypes']:
        commit_ft_dict = commit_stats['fileTypes'][file_type]
        if file_type in main_types:
            position = main_types.index(file_type)
            row[position] = commit_ft_dict['inserts']
type_df = pd.DataFrame(rows, columns=main_types) # index=commit_ids,
type_df

In [ ]:
type_corr = type_df.corr()
print(str(type_corr))

In [ ]:
for i in range(len(main_types)):
    for j in range(len(main_types)):
        if i < j:
            val = type_corr.iat[i, j]
            if val > 0.2:
                print(main_types[i] + ", " + main_types[j] + ": " + str(val))